<a href="https://colab.research.google.com/github/aguscura/Python-Deep-Learning/blob/main/CNN_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Ventana capa convolucional: 5x5

Ventana capa de pooling: 2x2

Filtros: 32

Activación: ReLU.

In [1]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D

model = Sequential()
model.add(Conv2D(32, (5,5), activation='relu', input_shape=(28,28,1)))
model.add(MaxPooling2D((2,2)))

In [2]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 24, 24, 32)        832       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 12, 12, 32)       0         
 )                                                               
                                                                 
Total params: 832
Trainable params: 832
Non-trainable params: 0
_________________________________________________________________


**832 parámetros.**

Como dijimos, cada filtro se compone de una ventana de (pesos + sesgo) de 5x5. En resumen, estamos agarrando una ventana de 5x5 de la imagen de entrada (de 28x28), le aplicamos este filtro mencionado y obtenemos la salida de 24x24.

Por eso la cantidad de 832 se explica así:

(5 x 5) + 1 (sesgo) = 26 parámetros para cada filtro

con 32 filtros:

parametros totales = 26 * 32 = 832.


La capa de pooling no tiene parametros entrenables porque siempre agarra el máximo.


In [3]:
# Hacemos nuestra red mas profunda, agregamos otra capa convolucional con 64 filtros mas y otra de pooling con ventana 2x2. 
# El input shape solo es necesario aclararlo en la primera capa. Luego keras lo deduce por nosotros

#Importante, ver que ahora la segunda capa convolucional tiene un tamaño de 8x8. Eso es porque la entrada de esta capa es la salida de la capa anterior (12x12).
#Luego, al tener una ventana deslizante de 5x5 y un stride o 'paso' de 1, la capa resultante es de 8x8. 

#Parámetros de la nueva capa:
#(Tamaño de la ventana * filtros de la capa anterior + sesgo ) * filtros de esta nueva capa
#       (((5x5)* 32) + 1 ) * 64 = 51264.

model.add(Conv2D(64, (5,5), activation='relu'))
model.add(MaxPooling2D((2,2)))

In [4]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 24, 24, 32)        832       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 12, 12, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 8, 8, 64)          51264     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 4, 4, 64)         0         
 2D)                                                             
                                                                 
Total params: 52,096
Trainable params: 52,096
Non-trainable params: 0
_________________________________________________________________


In [5]:
# Ahora sí, para hacer la clasificacion, conectamos una capa DENSAMENTE CONECTADA.
# Esta ultima capa SI VA A ESTAR CONECTADA A TODAS LAS NEURONAS DEL PASO ANTERIOR.

from tensorflow.keras import layers

model.add(layers.Flatten())
model.add(layers.Dense(10, activation='softmax'))

# IMPORTANTE

Antes de agregar la ultima capa densamente conectada con la funcion de activación softmax para obtener las probabilidades, tenemos que **APLANAR** su entrada.

El tensor de entrada a una capa final softmax es de 1D, mientras que la salida que teniamos luego del segundo maxpooling era un tensor 3D (4,4,64). 

Para eso, tenemos la capa flatten que nos permite obtener un vector de [1024] ( 4x4x64).

El número de parámetros de la capa densamente conectada será: 

TAMAÑO DE LA ENTRADA * PESOS + SESGOS (siendo pesos y sesgos igual al numero de neuronas de la capa densa de salida)

 (1024 * 10) + 10 = 10.250

In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 24, 24, 32)        832       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 12, 12, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 8, 8, 64)          51264     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 4, 4, 64)         0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 1024)              0         
                                                                 
 dense (Dense)               (None, 10)                1

# IMPORTANTE

Las redes neuronales convolucionales toman de entrada un tensor 3D (ancho, alto, canales (profundidad). 

Las redes densamente conectadas tomaban como entrada un vector de 1D. Por eso usabamos el flatten antes.

In [16]:
from tensorflow.keras.utils import to_categorical
import tensorflow as tf

mnist = tf.keras.datasets.mnist
(train_images, train_labels),(test_images, test_labels) = mnist.load_data()

train_images = train_images.reshape((60000,28,28,1)) #depth = 1 (escala de grises, no RGB)
test_images = test_images.reshape((10000,28,28,1))

train_images = train_images.astype('float32') / 255
test_images = test_images.astype('float32') / 255

print(train_labels)

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

model.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

model.fit(train_images, train_labels, verbose=1, epochs=5, batch_size=100)

test_loss, test_acc = model.evaluate(test_images, test_labels)

print('Test Accuracy:', test_acc)

[5 0 4 ... 5 6 8]
Epoch 1/5
600/600 [==============================] - 51s 85ms/step - loss: 0.8869 - accuracy: 0.7698
Epoch 2/5
600/600 [==============================] - 52s 87ms/step - loss: 0.2641 - accuracy: 0.9216
Epoch 3/5
600/600 [==============================] - 51s 85ms/step - loss: 0.1910 - accuracy: 0.9445
Epoch 4/5
600/600 [==============================] - 52s 87ms/step - loss: 0.1529 - accuracy: 0.9551
Epoch 5/5
313/313 [==============================] - 3s 8ms/step - loss: 0.1009 - accuracy: 0.9716
Test Accuracy: 0.9715999960899353
